In [6]:
!pip install torch_geometric
!pip install torch_sparse
!pip install torch_scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=279639 sha256=52fa239941b88c02f9b57af8a684089257efc608e922ef191323f56371e31676
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter


In [7]:
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.utils import dense_to_sparse
#from torch_geometric.nn.conv import TransformerConv


# ground truth
A = torch.tensor([
              [0.0, 1.0, 1.0, 0.0, 1.0],
              [1.0, 0.0, 1.0, 0.0, 0.0],
              [1.0, 1.0, 0.0, 1.0, 1.0],
              [0.0, 0.0, 1.0, 0.0, 0.0],
              [1.0, 0.0, 1.0, 0.0, 0.0]
])

#NUM_NODE = 5
#print(NUM_NODE)

# convert to input (edge indices)
#data = dense_to_sparse(A)

#print(data) # edge indices and edge attributes
#print(data[0])
#print(data[1])

# node feature
#B = torch.tensor([[0.0],[1.0],[2.0],[3.0],[4.0]])


# ground truth
A = torch.tensor([
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0],
[0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0],
[0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0],
[0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0],
[0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0],
[0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
],dtype=torch.float32)

NUM_NODE = 16
print(NUM_NODE)

# convert to input (edge indices)
data = dense_to_sparse(A)

print(data) # edge indices and edge attributes
print(data[0])
print(data[1])

# node feature
B = torch.tensor([[0.0],[1.0],[2.0],[3.0],[4.0],[5.0],
                  [6.0],[7.0],[8.0],[9.0],[10.0],
                  [11.0],[12.0],[13.0],[14.0],[15.0]])
#print(B)

# test case for MSE loss
#C = torch.clone(A)
#C[0,1] -= 0.5
#print(C[0,1])

#criterion = nn.MSELoss(reduction='sum')
#loss = criterion(C, A)

#print(loss)
#print(0.5**2)

16
(tensor([[ 2,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  6,  6,  7,  7,  7,  7,
          8,  8,  8,  8,  8,  9, 10, 10, 10, 10, 13, 15],
        [ 4,  5,  8, 10,  6,  7, 10,  2,  7, 10,  2,  8,  3,  7,  3,  4,  6, 10,
          2,  5,  9, 13, 15,  8,  2,  3,  4,  7,  8,  8]]), tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))
tensor([[ 2,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  6,  6,  7,  7,  7,  7,
          8,  8,  8,  8,  8,  9, 10, 10, 10, 10, 13, 15],
        [ 4,  5,  8, 10,  6,  7, 10,  2,  7, 10,  2,  8,  3,  7,  3,  4,  6, 10,
          2,  5,  9, 13, 15,  8,  2,  3,  4,  7,  8,  8]])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [10]:
def network_stat(network_y):
  n = network_y.shape[0]
  holder = torch.zeros(2)
  for i in range(n):
    for j in range(n):
      if i < j:
        holder[0] += network_y[i,j]
        for k in range(n):
          if j < k:
            holder[1] += network_y[i,k] * network_y[j,k]
  return(holder)

print(network_stat(A))

tensor([15., 10.])


In [11]:
import torch
from torch_geometric.data import DataLoader
from tqdm import tqdm
import numpy as np
#import mlflow.pytorch
#from utils import (count_parameters, gvae_loss, slice_edge_type_from_edge_feats, slice_atom_type_from_node_feats)
#from gvae import GVAE
#from config import DEVICE as device

import torch
import torch.nn as nn
from torch.nn import Linear
from torch_geometric.nn.conv import TransformerConv
from torch_geometric.nn import Set2Set
from torch_geometric.nn import BatchNorm
from tqdm import tqdm

from torch_geometric.nn import GAE, VGAE, GCNConv

In [17]:
class GVAE(nn.Module):
    def __init__(self, feature_size):
        super(GVAE, self).__init__()
        self.encoder_embedding_size = 8
        self.latent_embedding_size = 128
        self.decoder_hidden_neurons = 512

        # Encoder layers (1 is dimenstion of X)
        self.conv1 = GCNConv(1, 2 * self.encoder_embedding_size)
        self.conv2 = GCNConv(2 * self.encoder_embedding_size, self.encoder_embedding_size)

        # Pooling layers
        self.pooling = Set2Set(self.encoder_embedding_size, processing_steps=4)

        # Latent transform layers
        self.mu_transform = Linear(self.encoder_embedding_size*2, self.latent_embedding_size)
        self.logvar_transform = Linear(self.encoder_embedding_size*2, self.latent_embedding_size)

        # Decoder layers
        # --- Shared layers
        self.linear_1 = Linear(self.latent_embedding_size, self.decoder_hidden_neurons)
        self.linear_2 = Linear(self.decoder_hidden_neurons, self.decoder_hidden_neurons)

        # --- Edge decoding 
        edge_output_dim = int(NUM_NODE * (NUM_NODE-1) / 2)
        self.edge_decode = Linear(self.decoder_hidden_neurons, edge_output_dim)
        
    def reparameterize(self, mu, logvar):
        # Get standard deviation
        std = torch.exp(logvar)
        # Returns random numbers from a normal distribution
        eps = torch.randn_like(std)
        # Return sampled values
        return eps.mul(std).add_(mu)

    def encode(self, x, edge_index):
        # GNN layers
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()

        # Pool to global representation
        x = self.pooling(x)

        # Latent transform layers
        mu = self.mu_transform(x)
        logvar = self.logvar_transform(x)
        return mu, logvar


    def decode(self, z):
        #triu_logits = []
        
       # Pass through shared layers
        z = self.linear_1(z).relu()
        z = self.linear_2(z).relu()

        # Decode edge types
        edge_logits = self.edge_decode(z)
            
        return edge_logits

    def forward(self, x, edge_index):
        # Encode the molecule
        mu, logvar = self.encode(x, edge_index)
        # Sample latent vector (per atom)
        z = self.reparameterize(mu, logvar)
        # Decode latent vector into original molecule
        triu_logits = self.decode(z)

        return triu_logits, mu, logvar

 

def kl_loss(mu, logstd):
    """
    Closed formula of the KL divergence for normal distributions
    """
    MAX_LOGSTD = 10
    logstd =  logstd.clamp(max=MAX_LOGSTD)
    kl_div = -0.5 * torch.mean(torch.sum(1 + 2 * logstd - mu**2 - logstd.exp()**2, dim=1))

    # Limit numeric errors
    kl_div = kl_div.clamp(max=1000)
    return kl_div


def triu_to_dense(triu_values, num_nodes):
    """
    Converts a triangular upper part of a matrix as flat vector
    to a squared adjacency matrix with a specific size (num_nodes).
    """
    dense_adj = torch.zeros((num_nodes, num_nodes))
    triu_indices = torch.triu_indices(num_nodes, num_nodes, offset=1)
    #tril_indices = torch.tril_indices(num_nodes, num_nodes, offset=-1)
    dense_adj[triu_indices[0], triu_indices[1]] = triu_values
    dense_adj = torch.add(dense_adj, dense_adj.T)
    return dense_adj


def gvae_loss(triu_logits, mu, logvar, kl_beta, num_nodes, ground_truth, criterion):
    """
    Calculates the loss for the graph variational autoencoder,
    consiting of a node loss, an edge loss and the KL divergence.
    """
    recon = triu_to_dense(triu_logits, num_nodes)
    recon_loss = criterion(recon, ground_truth)

    # KL Divergence
    kl_divergence = kl_loss(mu, logvar)

    return recon_loss + kl_beta * kl_divergence

In [18]:
model = GVAE(feature_size=1)
criterion = nn.MSELoss(reduction='sum')



loss_fn = gvae_loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
kl_beta = 0.5



for epoch in range(1000): 
  #forward(self, x, edge_index)
  triu_logits, mu, logvar = model(B, data[0]) # edge_index is data[0], x is B (node feature)
  loss = loss_fn(triu_logits, mu, logvar, kl_beta, NUM_NODE, A, criterion) # A is ground truth adj matrix
  if epoch % 100 == 0: 
    print(loss)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


tensor(49.6895, grad_fn=<AddBackward0>)
tensor(1.9405, grad_fn=<AddBackward0>)
tensor(2.3966, grad_fn=<AddBackward0>)
tensor(0.5240, grad_fn=<AddBackward0>)
tensor(0.2523, grad_fn=<AddBackward0>)
tensor(0.4823, grad_fn=<AddBackward0>)
tensor(0.6623, grad_fn=<AddBackward0>)
tensor(0.2190, grad_fn=<AddBackward0>)
tensor(0.2820, grad_fn=<AddBackward0>)
tensor(0.2037, grad_fn=<AddBackward0>)


In [19]:
out = triu_to_dense(triu_logits, NUM_NODE)
out = (out>0.5).float()
print(network_stat(out))
print(network_stat(A))

tensor([15., 10.])
tensor([15., 10.])
